In [1]:
import pandas as pd

encoding = 'utf-8'

pd.set_option('max_colwidth',100)
pd.set_option('float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 15)

In [7]:
### read datasets : 

### 1. full datasets and only chose the publications between 1996 and 2021
### 2. mapping between author id and gender /discipline
### 3. mapping between pub id and citations

author_dis=pd.read_csv("...\\0_non_mover_gender_dis.csv",encoding=encoding)

papers = pd.read_parquet('...\\2021-08-11-scrp21-wholeworld_all_authorshiprecords.csv.sorted.parquet', engine='fastparquet')
papers_96_21=papers[(papers["pubyear"]<2022)&(papers["pubyear"]>1995)]


citation= pd.read_csv('...\\scp_B_rp_2020_ArRe_items_citations.csv',encoding=encoding)


paper_full=author_dis.merge(papers_96_21,how="left",on="author_id")
paper_full=paper_full.merge(citation,left_on="item_id",right_on="EID",how="left")


In [ ]:
### select the first pub for each author (published between 2012 and 2016)

author_first=paper_full.sort_values(by=["author_id","pubyear"]).groupby("author_id").head(1).copy()

author_12_16=author_first[(author_first["pubyear"]<=2016) & (author_first["pubyear"]>=2012)]
author_12_16["cohort"]=author_12_16["cohort"]
paper_author_12_16=paper_full[paper_index["author_id"].isin(author_12_16["author_id"])]

author_12_16_paper=author_12_16.merge(paper_index[["author_id","doi",'pubyear']],on="author_id",how="left")
author_12_16_early_paper=author_12_16_paper[((author_12_16_paper["pubyear"]-author_12_16_paper["cohort"]))<3]
author_12_16_early_paper=author_12_16_early_paper[["doi","pubyear"]].drop_duplicates()

## select the first pub as first author 
author_12_16_first=author_12_16_first[(author_12_16_first["doi"].notnull())&(author_12_16_first["sequence_nr"]==1)]

author_12_16_first=author_12_16_first[(author_12_16_first["gender"].notnull())&(author_12_16_first["discipline"].notnull())]


###  Using Altmetrics 
#### 1. get the number of original tweets, get the tweet_id_list, tweeter_id_list



In [ ]:
# 1 add altmetric

from pyaltmetric import Altmetric, Citation, HTTPException
import csv

altmetric_object = Altmetric(api_key = "***your Altmetric API***")

In [ ]:
# 1 add Tweety
import tweepy
import requests
# API keyws that yous saved earlier
Bearer_Token="***your Bearer_Token***"

Tweet_api_key = "***your Bearer_Token***"
api_secrets = "***your api_secrets***"
access_token = "***your access_token***"
access_secret = "***your access_secret***"
 
# Authenticate to Twitter
#auth = tweepy.OAuthHandler(api_key,api_secrets)
#auth.set_access_token(access_token,access_secret)
client = tweepy.Client(bearer_token=Bearer_Token , consumer_key=Tweet_api_key, consumer_secret=api_secrets,
                       access_token=access_token, access_token_secret=access_secret,return_type= requests.Response,
                      wait_on_rate_limit=True)


In [ ]:
headerList = [ 'doi','cited_by_posts_count', 'cited_by_tweeters_count','cited_by_fbwalls_count',
              "Al_year","Al_month","Al_day","Original_Tweets","Original_Tweet_list","Original_Tweeters","Original_Tweeter_list","details"]

In [ ]:
### Altmetric collection

In [ ]:
import collections

### used for getting the number of tweets
def unique_keyvalue(lst,key1,key2): 
    
    unique_counts = collections.Counter(e[key1][key2] for e in lst if key1 in e.keys() if  key2 in e[key1].keys())
    
    nunique_counts=len(unique_counts)
    
    return nunique_counts




def unique_key_2_values(lst,key1,key2): 
    
    unique_values =  list(set(dic[key1][key2] for dic in lst if key1 in dic.keys() if  key2 in dic[key1].keys()))
    #nunique_counts=len(unique_values)
    return unique_values

def unique_key_1_values(lst,key1):

    unique_values=[]
    unique_values =  list(set(dic[key1] for dic in lst if key1 in dic.keys()))
    
    return unique_values

def key_2_values(lst,key1,key2): 
    
    unique_values =  list(dic[key1][key2] for dic in lst if key1 in dic.keys() if  key2 in dic[key1].keys())
    counts=len(unique_values)
    return unique_values,counts

def key_1_values(lst,key1):

    unique_values=[]
    unique_values =  list(dic[key1] for dic in lst if key1 in dic.keys())
    
    return unique_values

In [ ]:


###
#  Function: find the time including the publish time
###
def fetch_first_time(response_json, type_time):

    result=[]
    #result.append (doi)
    if  type_time in response_json["citation"]: 
        date=response_json["citation"][type_time]

        result.append(int(date[0:4]))
        result.append(int(date[5:7]))
        result.append(int(date[8:10]))
    else: 
        result.append(0)
        result.append(0)
        result.append(0)
        
    return result


###
#  Function: find all the tweets and twitter
###
def fetch_tweet_tweeter( response_json):

    result=[]
    #result.append (doi) 

    if ('posts'in response_json): 
        if "twitter"in response_json["posts"]:
            twitter=response_json["posts"]["twitter"] 
        
            if len(twitter)>=2:               
                result.append(len(twitter))
                result.append(unique_key_1_values(twitter,"tweet_id"))
                result.append(unique_keyvalue(twitter,"author","tweeter_id"))
                result.append(unique_key_2_values(twitter,"author","tweeter_id"))
            else: 
                result.append(1)
                result.append(unique_key_1_values(twitter,"tweet_id"))
                result.append(1)
                result.append(unique_key_2_values(twitter,"author","tweeter_id"))
                    
        else:
            result.extend((0,[],0,[]))
    else:
        result.extend((0,[],0,[]))               
            
    return result


###
#  Function: find the unique tweets and twitter
###

def fetch_tweet_tweeter_nonunique( response_json):

    result=[]
    #result.append (doi) 

    if ('posts'in response_json): 
        if "twitter"in response_json["posts"]:
            twitter=response_json["posts"]["twitter"] 
        
            if len(twitter)>=2:
                tweeter_value,tweeter_count=key_2_values(twitter,"author","tweeter_id")
                result.append(len(twitter))
                result.append(key_1_values(twitter,"tweet_id"))
                result.append(tweeter_count)
                result.append(tweeter_value)

            else: 
                result.append(1)
                result.append(key_1_values(twitter,"tweet_id"))
                result.append(1)
                result.append(key_2_values(twitter,"author","tweeter_id"))
                    
        else:
            result.extend((0,[],0,[]))
    else:
        result.extend((0,[],0,[]))               
            
    return result


In [ ]:
output_16_withTW=output_16[(output_16["details"].notnull())&(output_16["Original_Tweets"]>0)]



In [ ]:



keys=['doi','cited_by_posts_count', 'cited_by_tweeters_count','cited_by_fbwalls_count',"details_url"]

def fetch_publishtime(doi):
    result=[]
    try: 
        x =doi.rstrip() 
        c = altmetric_object.doi(x)
        publish_time=datetime.datetime.fromtimestamp(c.get("pubdate"))
        result.append(doi)
        result.append(int(publish_time.strftime("%Y")))
        result.append(int(publish_time.strftime("%m")))
        result.append(int(publish_time.strftime("%d")))
    except:
        result=[doi,0,0,0]
    return result


def fake_info_media(x):

    result=[]
    try: 
        x =x.rstrip() 
        c = altmetric_object.doi(x)
        result.extend([c.get(key) for key in keys])
    except:
        result=[x,"None","None","None","None"]
    return result

 
def fake_info_tweet(doi):
    
    result=[doi]
    try: 
        req = Request(
                    url=url_prefix+doi+"?key=" +api_key+"&post_types=original_tweets",
                    headers={'User-Agent': 'Mozilla/5.0'}
                )
        response = urlopen(req, context=ctx)
        response_json=json.load(response)
        mm=1
        
    except:
        
        mm=0
    if mm==1:

        first_seen_time=fetch_first_time(response_json,"first_seen_on")
        
        published_on=fetch_first_time(response_json,"published_on")

        Tweet_tweeter=fetch_tweet_tweeter(response_json)
        result=result+published_on+first_seen_time+Tweet_tweeter
    
    return result
    
def fake_info_tweet_complete(doi):
    
    result=[doi]
    try: 
        req = Request(
                    url=url_prefix+doi+"?key=" +api_key+"&post_types=original_tweets",
                    headers={'User-Agent': 'Mozilla/5.0'}
                )
        response = urlopen(req, context=ctx)
        response_json=json.load(response)
        mm=1
        
    except:
        
        mm=0
    if mm==1:
        first_seen_time=fetch_first_time(response_json,"first_seen_on")
        
        published_on=fetch_first_time(response_json,"published_on")
        Tweet_tweeter=fetch_tweet_tweeter_nonunique(response_json)
        result=result+published_on+first_seen_time+Tweet_tweeter
    
    return result



In [ ]:
### parallelize the collection process (

from pathos.pp import ParallelPool
from pathos.threading import ThreadPool

In [ ]:

#pool = ParallelPool(nodes=4)

def function_called_by_starmap(doi):
    print( doi)
    return doi


def multiprocessing_api_retrieve_all_query_results(input_doi) -> list:
    # Multiprocessing module to split the work between processes and gather it up at the end
    out_put_x = []
    out_put_y = []


    
    # Start one process per cpu core
    #with Pool(2) as p:
    with ThreadPool(nodes=1) as p:

        # More on starmap() below
        #completed_x = p.map(function_called_by_starmap, input_doi)
        completed_x = p.map(fake_info_media, input_doi)
        completed_y = p.map(fake_info_tweet_complete, input_doi)
        #while not completed_x.ready():
            #time.sleep(5)
            #print("1")
            
    # Add the request results completed in parallel to the result list
    out_put_x.extend(completed_x)
    out_put_y.extend(completed_y)
    
    return out_put_x,out_put_y


In [ ]:
headerList_x = [ 'doi','cited_by_posts_count', 'cited_by_tweeters_count','cited_by_fbwalls_count',"details"]
headerList_y = [ 'doi',"Publish_year","Publish_month","Publish_day","Al_year","Al_month","Al_day",
                "Original_Tweets","Original_Tweet_list","Original_Tweeters","Original_Tweeter_list"]
input_doi_all=list(input_all["doi"].unique())

## n for blocks 
n=int(len(input_doi_all)/1000)



for i in range (n):
    if (i < n):
        input_doi=input_doi_all[(i*1000):(i*1000+1000)] 
    else:
        input_doi=input_doi_all[(i*1000):] 
        
    ## x: social data csv; y: tweet data csv
    x,y = multiprocessing_api_retrieve_all_query_results(input_doi)
   
    if (i>0):
        out_put_x=concat(out_put_x,pd.DataFrame(x))
        out_put_y=concat(out_put_y,pd.DataFrame(y))
    else:
        out_put_x=pd.DataFrame(x)
        out_put_y=pd.DataFrame(y)
    
out_put_x.columns=headerList_x
out_put_y.columns=headerList_y
out_put_x=out_put_x.reset_index()
out_put_y=out_put_y.reset_index()
out_put_all=out_put_x.merge(out_put_y,on="index",how="left")

out_put_all.to_pickle("...\\result\\doi_3y_2012_2016_tweet_all.csv")


### using Twitter API 
#### 2. select the origin tweets in the first month ----> create_at

#### 3. identify the self_prmotion ----> name

In [ ]:
for i in range (n):
    #i=i+3
    input_data=out_put_all[(i*1000):(i*1000+1000)]
    input_data.columns=headerList_y
    input_tweet_time=input_data[(input_data["Original_Tweets"].notnull())&(input_data["Original_Tweets"]!=0)]
    input_tweet_time=input_tweet_time[["doi","Original_Tweet_list"]].explode("Original_Tweet_list")
    input_tweet_time_1=input_tweet_time.drop_duplicates("Original_Tweet_list")

    

    input_tweet_time_block=[]
   
    #for i in range(round(len(df)/5)): #This ensures all rows are captured
    for m in range (int(np.ceil(input_tweet_time_1.shape[0] / 50))):
        input_tweet_time_block.append(
        {
            #'doi_list': input_tweet_time_1.reset_index().loc[m*50:(m+1)*50,:]["doi"].tolist(),
            'Original_Tweet_list': input_tweet_time_1.reset_index().loc[m*50:(m+1)*50,:]["Original_Tweet_list"].tolist(),

        }
    )
  
    start_time=time.time()
    print(i," start time: ", start_time, " n_block: ", m)
    input_tweet_time_block=pd.DataFrame(input_tweet_time_block)
    input_tweet_time_block["Original_Tweet_list"]=input_tweet_time_block["Original_Tweet_list"].apply(lambda x:[y for y in x if type(y)==str])
    out_put_x=get_time_name_df(input_tweet_time_block)
    input_tweet_time=input_tweet_time.merge(out_put_x,on="Original_Tweet_list",how="left")
    input_tweet_time_2=input_tweet_time.groupby('doi').agg({'Original_Tweet_list':lambda x: list(x),
                                                            'text':lambda x: list(x),
                                                            'twtime_list':lambda x: list(x),
                                                            'Original_Tweeter_list':lambda x: list(x), 
                                                            'tw_name_list':lambda x: list(x),
                                                            'tw_handle_list':lambda x: list(x),
                                                            "account_time":lambda x: list(x)})     
    input_data=input_data.merge(input_tweet_time_2,on="doi",how="left")
    
    
    if (i>0):
        tweet_info=concat(tweet_info,input_data)

    else:
        tweet_info=input_data
    
    print (i,": ", (time.time()-start_time))
    time.sleep(60)
    
    
    
tweet_info=tweet_info.drop(columns=["Original_Tweet_list_x","Original_Tweeter_list_x"]).rename(columns={"Original_Tweet_list_y":"Original_Tweet_list","Original_Tweeter_list_y":"Original_Tweeter_list"})
tweet_info.to_pickle("....\\doi_tweet_info_all.csv")    